<a href="https://colab.research.google.com/github/christyesmee/Thesis/blob/main/Label_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive_path = "/content/drive"
drive.mount(drive_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import os
import torchvision.models as models
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision.transforms import CenterCrop
from torchvision.models.resnet import ResNet18_Weights
from tqdm.notebook import tqdm, trange

# NDVI calculations

In [ ]:
# File path setup
img_dir = "18_sat_pt_processed"
folder_path = f"{drive_path}/MyDrive/{img_dir}"

# Image path & scale
file_name = "galicia_split_image"
file_path = f"{folder_path}/{file_name}"
scale = 10  # pixels per meter
AS_TYPE = torch.float32

In [ ]:
def calculate_ndvi_for_image(data):
    """
    Calculate NDVI for each pixel in the image.

    Parameters:
    data (torch.Tensor): Tensor containing the image data with shape (num_channels, height, width)

    Returns:
    torch.Tensor: NDVI values for each pixel in the image with shape (height, width)
    """
    # Extract the NIR and RED bands (assuming specific indices, adjust if necessary)
    nir_band = data[3, :, :]  # NIR band, example index 3
    red_band = data[0, :, :]  # RED band, example index 0

    # Initialize the NDVI tensor
    ndvi_band = torch.zeros_like(nir_band)

    # Get the height and width of the image
    height, width = nir_band.shape

    # Calculate NDVI for each pixel
    epsilon = 1e-6  # Small value to avoid division by zero
    for row in range(height):
        for col in range(width):
            nir_value = nir_band[row, col].item()
            red_value = red_band[row, col].item()
            ndvi_value = (nir_value - red_value) / (nir_value + red_value + epsilon)
            ndvi_band[row, col] = ndvi_value

    return ndvi_band

In [ ]:
def calculate_mean_ndvi(ndvi_path):
    """
    Calculate the mean NDVI value for the image stored in the .pt file by looping through all pixel values.

    Parameters:
    ndvi_path (str): Path to the .pt file containing the NDVI image

    Returns:
    float: Mean NDVI value
    """
    # Load the NDVI image from the .pt file
    ndvi_image = torch.load(ndvi_path)

    # Initialize a list to hold all valid NDVI values
    ndvi_values = []

    # Get the height and width of the image
    height, width = ndvi_image.shape

    # Loop through each pixel in the image to collect NDVI values
    for row in range(height):
        for col in range(width):
            ndvi_value = ndvi_image[row, col].item()
            if torch.isfinite(torch.tensor(ndvi_value)):  # Check if the value is finite (not NaN or Inf)
                ndvi_values.append(ndvi_value)

    # Calculate the mean NDVI value
    if len(ndvi_values) > 0:
        mean_ndvi = sum(ndvi_values) / len(ndvi_values)
    else:
        mean_ndvi = float('nan')  # If no valid values, return NaN

    return mean_ndvi

In [ ]:
# calcualte NDVI for each pixel for every image

slices = 7

for idx in trange(slices):
    for jdx in trange(slices, leave=False):

        # Load tensor as array
        file_path_full = f"{file_path}_{idx}_{jdx}.pt"
        data = torch.load(file_path_full)

        try:
            ndvi_image = calculate_ndvi_for_image(data)

            # Print NDVI values for some example pixels
            print(f'{idx}_{jdx} -- NDVI value at (100, 200): {ndvi_image[100, 200].item()}')

            # Save the NDVI image as a .pt file
            output_path = f"{folder_path}/ndvi18_{idx}_{jdx}.pt"
            torch.save(ndvi_image, output_path)
            print(f'{idx}_{jdx} -- NDVI image saved to {output_path}')

        except:
            print(f'{idx}_{jdx} -- File not found: {idx}_{jdx}')

In [ ]:
from tqdm import trange

slices = 7

for idx in trange(slices):
    if idx < 5:
        continue  # Skip all indices before 5

    for jdx in trange(slices, leave=False):
        if idx == 5 and jdx < 3:
            continue  # Skip all jdx indices before 3 when idx is 5

        # Load tensor as array
        file_path_full = f"{file_path}_{idx}_{jdx}.pt"
        try:
            data = torch.load(file_path_full)
            ndvi_image = calculate_ndvi_for_image(data)

            # Print NDVI values for some example pixels
            print(f'{idx}_{jdx} -- NDVI value at (100, 200): {ndvi_image[100, 200].item()}')

            # Save the NDVI image as a .pt file
            output_path = f"{folder_path}/ndvi1819_{idx}_{jdx}.pt"
            torch.save(ndvi_image, output_path)
            print(f'{idx}_{jdx} -- NDVI image saved to {output_path}')

        except FileNotFoundError:
            print(f'{idx}_{jdx} -- File not found: {file_path_full}')
        except Exception as e:
            print(f'{idx}_{jdx} -- Error processing file: {str(e)}')


 57%|█████▋    | 4/7 [02:26<01:49, 36.60s/it]

5_3 -- NDVI value at (100, 200): 0.5661264061927795
5_3 -- NDVI image saved to /content/drive/MyDrive/Thesis Esmee/processed/1819_sat/ndvi1819_5_3.pt
5_4 -- NDVI value at (100, 200): 0.516076922416687



 71%|███████▏  | 5/7 [04:41<02:05, 62.57s/it]

5_4 -- NDVI image saved to /content/drive/MyDrive/Thesis Esmee/processed/1819_sat/ndvi1819_5_4.pt



 86%|████████▌ | 6/7 [06:54<01:21, 81.64s/it]

...

6_4 -- NDVI value at (100, 200): 0.5310978293418884
6_4 -- NDVI image saved to /content/drive/MyDrive/Thesis Esmee/processed/1819_sat/ndvi1819_6_4.pt



 86%|████████▌ | 6/7 [13:32<02:16, 136.14s/it]

6_5 -- NDVI value at (100, 200): 0.3225417137145996
6_5 -- NDVI image saved to /content/drive/MyDrive/Thesis Esmee/processed/1819_sat/ndvi1819_6_5.pt



100%|██████████| 7/7 [24:53<00:00, 213.34s/it]

6_6 -- NDVI value at (100, 200): 0.3148445785045624
6_6 -- NDVI image saved to /content/drive/MyDrive/Thesis Esmee/processed/1819_sat/ndvi1819_6_6.pt


In [ ]:
# calculating mean NDVI for one image

slices = 7
ndvi1718_list = []

for idx in trange(slices):
    for jdx in trange(slices, leave=False):

        # Load tensor as array
        ndvi_file_path = f"{folder_path}/ndvi1718_{idx}_{jdx}.pt"
        data = torch.load(ndvi_file_path)

        try:
            # Calculate the mean NDVI value for the image
            mean_ndvi_value = calculate_mean_ndvi(ndvi_file_path)
            print(f'{idx}_{jdx} -- Mean NDVI value: {mean_ndvi_value}')
            ndvi1718_list.append(mean_ndvi_value)                            # Or store in tensor?
            print(ndvi1718_list)
            print(f'{idx}_{jdx} -- Finished')

        except:
            print(f'{idx}_{jdx} -- File not found: {idx}_{jdx}')

In [ ]:
# calculate ndvi for all images

# we now have a list of values from all 49 tensors from 1718.
# we need the mean ndvi for the entire area, so we have to get the mean of all 49 tensors

ndvi1718_label = sum(ndvi1718_list) / len(ndvi1718_list)
print(f'ndvi mean 1718{ndvi1718_label}')

In [ ]:
# Calculate Ground Truth NDVI Difference:
# now we need to calculate the difference between 1819 and 1718 labels for each pixel



In [ ]:
import torch

# Load the tensor
output_path = '/content/drive/MyDrive/Thesis Esmee/processed/1718_sat/ndvi_1718_0_0.pt'
tensor = torch.load(output_path)

# Print the shape of the tensor
print(tensor.shape)

# Print a subset of the tensor values
subset = tensor[:10, :10]  # Adjust the indices as needed
print("Subset of tensor values:")
print(subset)


torch.Size([2400, 2400])
Subset of tensor values:
tensor([[   nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
            nan],
        [0.3562, 0.3583, 0.3498, 0.3498, 0.3656, 0.3649, 0.3461, 0.3461, 0.3685,
         0.3521],
        [0.3788, 0.3745, 0.3762, 0.3762, 0.3888, 0.3692, 0.3381, 0.3381, 0.3406,
         0.3338],
        [0.3960, 0.4028, 0.4105, 0.4105, 0.4045, 0.3728, 0.3337, 0.3337, 0.3160,
         0.3119],
        [0.3930, 0.4091, 0.4067, 0.4067, 0.3951, 0.3559, 0.3158, 0.3158, 0.3021,
         0.2951],
        [0.3943, 0.4080, 0.3963, 0.3963, 0.3781, 0.3332, 0.3126, 0.3126, 0.3124,
         0.2973],
        [0.4156, 0.4219, 0.4030, 0.3698, 0.3698, 0.3331, 0.3345, 0.3345, 0.3431,
         0.3506],
        [0.4195, 0.4243, 0.4033, 0.3649, 0.3649, 0.3596, 0.3909, 0.3909, 0.4137,
         0.4493],
        [0.4253, 0.4244, 0.3899, 0.3708, 0.3708, 0.3902, 0.4295, 0.4295, 0.4612,
         0.4685],
        [0.4106, 0.3831, 0.3775, 0.4139, 0.4139, 0.4260, 0.

In [ ]:
import torch

def calculate_mean_ndvi(ndvi_path):
    """
    Calculate the mean NDVI value for the image stored in the .pt file by looping through all pixel values.

    Parameters:
    ndvi_path (str): Path to the .pt file containing the NDVI image

    Returns:
    float: Mean NDVI value
    """
    # Load the NDVI image from the .pt file
    ndvi_image = torch.load(ndvi_path)

    # Initialize a list to hold all valid NDVI values
    ndvi_values = []

    # Get the height and width of the image
    height, width = ndvi_image.shape

    # Loop through each pixel in the image to collect NDVI values
    for row in range(height):
        for col in range(width):
            ndvi_value = ndvi_image[row, col].item()
            if torch.isfinite(torch.tensor(ndvi_value)):  # Check if the value is finite (not NaN or Inf)
                ndvi_values.append(ndvi_value)

    # Calculate the mean NDVI value
    if len(ndvi_values) > 0:
        mean_ndvi = sum(ndvi_values) / len(ndvi_values)
    else:
        mean_ndvi = float('nan')  # If no valid values, return NaN

    return mean_ndvi

# Path to the NDVI image .pt file
ndvi_file_path = '/content/drive/MyDrive/Thesis Esmee/processed/1718_sat/ndvi_1718.pt'

# Calculate the mean NDVI value for the image
mean_ndvi_value = calculate_mean_ndvi(ndvi_file_path)

print(f'Mean NDVI value: {mean_ndvi_value}')


Mean NDVI value: 0.44347531549492564


In [ ]:
# now calculate for all images by looping through the 1718_sat dict for all .pt
# return first all images ndvi means
# then take the average of those 49 images
# this average is the ndvi mean of 1718

# then do the same for 1819


# then 18mean(for each image/pixel) - 17mean(for each image/pixel) = ndvi_target (to train with 1718)

In [ ]:
import torch

def calculate_ndvi_for_image(data):
    """
    Calculate NDVI for each pixel in the image.

    Parameters:
    data (torch.Tensor): Tensor containing the image data with shape (num_channels, height, width)

    Returns:
    torch.Tensor: NDVI values for each pixel in the image with shape (height, width)
    """
    # Extract the NIR and RED bands (assuming specific indices, adjust if necessary)
    nir_band = data[3, :, :]  # NIR band, example index 3
    red_band = data[0, :, :]  # RED band, example index 0

    # Initialize the NDVI tensor
    ndvi_band = torch.zeros_like(nir_band)

    # Get the height and width of the image
    height, width = nir_band.shape

    # Calculate NDVI for each pixel
    epsilon = 1e-6  # Small value to avoid division by zero
    for row in range(height):
        for col in range(width):
            nir_value = nir_band[row, col].item()
            red_value = red_band[row, col].item()
            ndvi_value = (nir_value - red_value) / (nir_value + red_value + epsilon)
            ndvi_band[row, col] = ndvi_value

    return ndvi_band

# Load the tensor from the file
file_path = '/content/drive/MyDrive/Thesis Esmee/processed/1718_sat/galicia_split_image_0_0.pt'
data = torch.load(file_path)

# Calculate NDVI for the image
ndvi_image = calculate_ndvi_for_image(data)

# Print NDVI values for some example pixels
print(f'NDVI value at (100, 200): {ndvi_image[100, 200].item()}')
print(f'NDVI value at (150, 250): {ndvi_image[150, 250].item()}')


NDVI value at (100, 200): 0.3452032804489136
NDVI value at (150, 250): 0.4811120331287384
